<a href="https://colab.research.google.com/github/salim4n/Mes-Notebooks/blob/main/telegram_callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
import requests
from datetime import datetime

class TelegramCallback(tf.keras.callbacks.Callback):
    def __init__(self, bot_token, chat_id, send_interval=1):
        """
        Initialise le callback Telegram

        Args:
            bot_token (str): Token du bot Telegram
            chat_id (str): ID du chat où envoyer les messages
            send_interval (int): Envoyer un message tous les N epochs
        """
        super().__init__()
        self.bot_token = bot_token
        self.chat_id = chat_id
        self.send_interval = send_interval
        self.base_url = f"https://api.telegram.org/bot{bot_token}/sendMessage"

    def send_message(self, message):
        """Envoie un message via l'API Telegram"""
        payload = {
            'chat_id': self.chat_id,
            'text': message,
            'parse_mode': 'HTML'
        }
        try:
            response = requests.post(self.base_url, json=payload)
            response.raise_for_status()
        except Exception as e:
            print(f"Erreur lors de l'envoi du message Telegram: {e}")

    def on_train_begin(self, logs=None):
        """Appelé au début de l'entraînement"""
        start_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        message = f"🚀 <b>Début de l'entraînement</b>\n"
        message += f"Heure de début: {start_time}"
        self.send_message(message)

    def on_epoch_end(self, epoch, logs=None):
        """Appelé à la fin de chaque epoch"""
        logs = logs or {}

        # N'envoyer que tous les N epochs
        if (epoch + 1) % self.send_interval != 0:
            return

        # Créer le message avec les métriques
        message = f"📊 <b>Epoch {epoch + 1}</b>\n\n"

        for metric, value in logs.items():
            message += f"{metric}: {value:.4f}\n"

        self.send_message(message)

    def on_train_end(self, logs=None):
        """Appelé à la fin de l'entraînement"""
        end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        message = f"✅ <b>Entraînement terminé</b>\n"
        message += f"Heure de fin: {end_time}"
        self.send_message(message)

In [ ]:

telegram_callback = TelegramCallback(
    bot_token='your-bot-token',
    chat_id='your-chat-id',
    send_interval=1
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)


model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[telegram_callback]
)